In [1]:
import json
import os
import pandas as pd

In [ ]:
prod_activ = pd.read_csv('products_activity.csv', on_bad_lines='skip')
prod_activ['BGC_id'] = ''
print(prod_activ)

In [ ]:
BGCs_with_activity_list = []

for dirpath, dirnames, files in os.walk('../'):
    for file_name in files:
        if file_name.endswith('json'):
            with open(file_name,'r') as json_read:
                print(file_name)
                BGC_id = file_name.replace('.json','')
                bgc_json = json.load(json_read)
                comp = bgc_json['cluster']['compounds']
                for i in comp:
                    product = i['compound']
                    print(product)
                    product_ = product.replace('\'', '').replace('β', '_beta').split(' ')[0]                 
                    print(product_)
                    try:
                        const = prod_activ.loc[(prod_activ['cluster_name']).str.contains(str(product_))]
                        if const.empty == False:
                            gbk_file_name = file_name.replace('.json','.gbk')
                            BGCs_with_activity_list.append(gbk_file_name)
                        else:
                            pass                   
                    except:
                        print("Error")

                    try:
                        prod_activ.loc[(prod_activ['cluster_name']).str.contains(str(product_)), ['BGC_id']] = BGC_id
                    except:
                        print('Error2')
                        
print(prod_activ)

In [7]:
pfam_out = pd.read_csv('../pfam.tsv', on_bad_lines='skip', delimiter='\t')

pfam_out['BGC_id'] = pfam_out['query name'].str.split('|').str[0]

queries = pfam_out['query name'].str.split('|').str[0].to_list()

In [ ]:
pd.set_option('display.min_rows', 150)

pfam_acc = pfam_out[['query name', 'accession','BGC_id']]

pf2 = pfam_acc.groupby('query name').first()

In [ ]:
## Descomentar a linha a seguir caso ainda não exista a pasta 'data_frame_00'

#os.mkdir('data_frame_00')

for i in pf2['BGC_id'].unique():
    a1 = pf2.loc[pf2['BGC_id'] == i]
    
    print(len(pf2['BGC_id'].unique()))
    
    values = a1['accession'].value_counts().keys().tolist()
    counts = a1['accession'].value_counts().tolist()
    
    PFAM_ID = {}
    for keys in values:
        for count in counts:
            PFAM_ID[keys] = count
            counts.remove(count)
            break
    
    PFAM_ID_df = pd.DataFrame([PFAM_ID])
    PFAM_ID_df['BGC_id'] = i
    PFAM_ID_df.to_csv('data_frame_00/'+i+'tempfile_PFAM_ID.csv')

files_paths = []
check = 'checkpoint'

for dirpath, dirnames, files in os.walk('./'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if file_name.startswith('BGC') and file_name.endswith('.csv') and check not in file_name:
            files_paths.append(file_path)

df1 = pd.concat(map(pd.read_csv, files_paths), ignore_index=False)
df1 = df1.fillna(0)
df1.to_csv('data_frame_00/all_features_PFAM_ID.csv', header=True)